In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! unzip /content/gdrive/My\ Drive/anime\ face\ generation/faces.zip

Archive:  /content/gdrive/My Drive/anime face generation/faces.zip
replace faces/118x118_to_127x127/1000565char4_face0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
import sys 
sys.path.append('/content/gdrive/My Drive/i2v')
import i2v #https://github.com/rezoo/illustration2vec
from PIL import Image
import numpy as np
!mkdir features


mkdir: cannot create directory ‘features’: File exists


In [ ]:
illust2vec = i2v.make_i2v_with_chainer(
    "/content/gdrive/My Drive/i2v/illust2vec_tag_ver200.caffemodel", "/content/gdrive/My Drive/i2v/tag_list.json")

In [ ]:
tag_dir = '/content/tags.txt'
def load_tags(tag_dir = tag_dir):
  tags = open(tag_dir , 'r')
  tags = tags.read().split('\n')
  print(tags)
  result = []
  for tag in tags:
    words = ''
    tag = tag.split()
    words += tag[0]
    for i in range(0, int(len(tag) - 1)):
      words += ' '
      words += tag[i+1]
    result.append(words)
  print("{} tags are loaded".format(len(result)))
  return result
tags = load_tags()

['long hair ', 'short hair', 'twintails', 'drill hair ', 'ponytail', 'blonde hair', 'brown hair ', 'black hair ', 'blue hair ', 'pink hair ', 'purple hair  ', 'green hair ', 'red hair', 'orange hair', 'silver hair', 'white hair  ', 'hair ribbon', 'hairband', 'hairclip', 'aqua hair ', 'multicolored hair', 'hair flower ', 'blue eyes ', 'red eyes', 'brown eyes', 'green eyes', 'closed eyes', 'yellow eyes', 'pink eyes', 'aqua eyes', 'black eyes', 'orange eyes', 'grey eyes', 'open mouth', 'ribbon', 'blush', 'smile', 'light smile', 'school uniform']
39 tags are loaded


In [ ]:
import os
def get_images(dir):
  folders = os.listdir(dir)
  images = {}
  for folder in folders: 
    images[folder]  = os.listdir(dir+ '/' +folder)
  return images

In [ ]:
def path_to_name(path):
    name = path.split('/')
    name = name[-1]
    name = name.split('.')
    name = name[0]
    return name

In [ ]:
save_dir = '/content/features'
processed = 0
def one_image(path , save_dir = save_dir , i2v = illust2vec, tags = tags ):
    name = path_to_name(path)
    file = '{}/{}_{}'.format(save_dir,name,'feature')
    global processed 
    if os.path.isfile(file + '.npy'):
      processed += 1 
      return 
    img = Image.open(path)
    feature  = illust2vec.estimate_specific_tags([img], tags)
    feature = feature[0]
    vector = []
    for tag in feature:
      vector.append(feature[tag])
    vector = np.asarray(vector)
    np.save( file ,vector)
    
    processed +=1 
    return vector

In [ ]:
import random
def folder_to_paths(dir):
    dic = get_images(dir)
    paths = []
    for folder in dic:
        names = dic[folder]
        pths = ['{}/{}/{}'.format(dir,folder,name) for name in names]
        paths.extend(pths)
    random.shuffle(paths)
    return paths

In [ ]:
paths = folder_to_paths("/content/faces")

In [ ]:
one_image(paths[187])

array([3.00424993e-02, 6.17325783e-01, 8.38140547e-02, 6.04176521e-03,
       2.04521418e-03, 6.73348188e-01, 1.63984597e-02, 1.66147947e-04,
       1.15334988e-04, 5.87430596e-03, 5.33223152e-04, 9.06586647e-05,
       6.34860992e-03, 2.95558572e-02, 6.39557838e-05, 1.06245279e-04,
       9.76147950e-02, 4.75299209e-01, 5.79851866e-03, 2.95042992e-06,
       1.15305185e-03, 2.54604220e-03, 5.81324100e-04, 8.08332205e-01,
       1.69509947e-02, 2.83271074e-04, 4.20153141e-04, 2.61211395e-03,
       2.47376561e-02, 5.00977039e-05, 2.12341547e-04, 5.50979376e-03,
       1.31249428e-04, 2.49269605e-02, 1.65058345e-01, 4.97253239e-01,
       6.23940110e-01, 1.07573867e-02, 7.17529058e-02])

In [ ]:
failed = []
def paths_to_np(paths):
    global failed
    for path in paths:
      try:
        one_image(path) 
      except Exception as err: 
        print('Exception when processing {}'.format(name(path)))
        print(err.args)
        print(err)
        print(type(err))
        failed.append(path)
    return


In [ ]:
import time
import threading
def multiprocessing(dir, num_threads):
  paths = folder_to_paths(dir)
  total = int(len(paths))
  print('{} pictures will be processed'.format(total))
  path_per_process = int(len(paths)/num_threads)
  threads = []
  starting = time.time()
  for i in range(int(num_threads +1 )):
    thread = threading.Thread(target = paths_to_np , args = ([paths[i*path_per_process:(i+1)*path_per_process]]))
    thread.start()
    threads.append(thread)

  while True: 
    time.sleep(5)
    sequence_finished = True
    for thread in threads: 
      sequence_finished &=  not thread.is_alive()
    if(sequence_finished):
      print('processing completed {}: processed'.format(processed))
      break
    else: 
      print('{} processed {} remaining {}% completed'.format(processed, total - processed, processed / total * 100))

  finishing = time.time()
  print('finished in {} seconds'.format(finishing - starting))
  return

In [ ]:
processed = 0
dir = '/content/faces'
multiprocessing(dir, 100)

In [ ]:
!zip -r /content/getchu_features.zip /content/features